# Scraping of location and details using Google Maps API

## Safe Spots

Code here demonstrates collecting details of Delhi hospitals. You'll need to get an API key from https://developers.google.com/places/web-service/intro.


## Google Maps API

The API allows search to be scripted both by -
1. Specifying relevant name of the location
2. Specifying the location co-ordinates (latitude, longitude) as well as the radius from the location within which the spots need be to located 

Since details for a number of cities required to be scraped, the former format was followed here. The request response is in JSON format. 
#### Google pins a "place ID" to every location embedded in its maps. These ID's were first collected, and later searched in Google Places Search to gather the details.

url="https://maps.googleapis.com/maps/api/place/textsearch/json?query=delhi+hospital&key="+key


In [3]:
import pandas as pd
import requests

key="<insert your Google Maps API key here>"

i=0

url="https://maps.googleapis.com/maps/api/place/textsearch/json?query=delhi+hospital&key="+key


#### The limit set by Google on free scraping of its search output is 60 results. Requests per week are also restricteed per key.

A single page displays 20 search results, hence the token for the next page found in 'next_page_token' key in the JSON needs to be extracted and appended to the URL to move on to next page.

(I tried putting this in a loop but the requests didn't seem to work, so I've delineated the step-by-step iterations)

In [4]:

# PAGE RESULT - ONE
# Search Results 1-20
r=requests.get(url)
out=r.json()

# Extract the place ID's from the dictionary so that their details can be scraped
id=[d['place_id'] for d in out['results']] 
nextpagetoken=[]
    
nextpagetoken=out['next_page_token']

# URL for the second page
url="https://maps.googleapis.com/maps/api/place/textsearch/json?query=delhi+hospital&key="+key+"&pagetoken="+nextpagetoken

#PAGE RESULT - TWO
#Search Results 21-40
r=requests.get(url)
out=r.json()

id1=[d['place_id'] for d in out['results']] 
nextpagetoken=[]
    
nextpagetoken=out['next_page_token']

# URL for the third page
url="https://maps.googleapis.com/maps/api/place/textsearch/json?query=delhi+hospital&key="+key+"&pagetoken="+nextpagetoken

# PAGE RESULT - THREE
# Search Results 41-60
r=requests.get(url)
out=r.json()
id2=[d['place_id'] for d in out['results']]


KeyError: 'next_page_token'

Now, to make a list of all the 60 place ID's we've gathered.

In [ ]:

id=id+id1+id2


### Extracting necessary information from the Google Maps search result

For the Safecity purpose, the details scraped were - 
1. Name of the police station/hospital
2. Address
3. Location co-ordinates (Latitude, Longitude)
4. Contact Number
5. Website 
6. Google Rating

Details for a particular location embedded in Google Maps can be found by using its place ID :

url="https://maps.googleapis.com/maps/api/place/details/json?placeid="+d+"&key="+key


In [ ]:

contact=[]
website=[]
rating=[]
address=[]
name=[]
lat=[]
lng=[]

for d in id:
    url="https://maps.googleapis.com/maps/api/place/details/json?placeid="+d+"&key="+key
    r=requests.get(url)
    output=r.json()
    
    
    # Collecting the Name, Address, Latitude, Longitude
    name.append(output['result']['name'])
    address.append(output['result']['formatted_address'])
    lat.append(output['result']['geometry']['location']['lat'])
    lng.append(output['result']['geometry']['location']['lng'])
    
     
    # Collecting Contact, Website, Google Rating
    if 'international_phone_number' in output['result'].keys():
        contact.append(output['result']['international_phone_number'])
    else:
        contact.append("Not Available")
        
    if 'website' in output['result'].keys():
        website.append(output['result']['website']) #get website
    else:
        website.append("Not Available")
    
    if 'rating' in output['result'].keys():
        rating.append(output['result']['rating'])
    else:
        rating.append("Not Available")



And the final step, saving to a dataframe and onto a csv file.

In [ ]:
       
df=pd.DataFrame()
df["NAME"]=name
df["ADDRESS"]=address
df["LATITUDE"]=lat
df["LONGITUDE"]=lng
df["CONTACT No."]=contact
df["WEBSITE"]=website
df["GOOGLE RATING"]=rating
       
df.to_csv("Safespots_DelhiHospitals.csv",index=None)

#### The catch here is that often the same location/spot could appear as multiple results in our search, since they have multiple embeddings on the map. 
For example, 'Fortis Hospitals', 'Fortis Hospital', 'Fortis Hospital Gurgaon', 'Fortis Hospital Gurugram' etc. These would need to be proofread/validated.